In [2]:
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')  # Para usar um banco de dados temporário na memória
cur = con.cursor()

CRIANDO AS TABELAS

In [3]:
cur.execute('''
CREATE TABLE Cliente (
    CPF INT PRIMARY KEY,
    Nome VARCHAR(100),
    Telefone VARCHAR(11),
    Data_Nascimento DATE
);
''')

cur.execute('''
CREATE TABLE Festa (
    ID_Festa INT PRIMARY KEY,
    Nome_Festa VARCHAR(100),
    Data DATE,
    Atracoes VARCHAR(255)
);
''')

cur.execute('''
CREATE TABLE Funcionario (
    ID_Funcionario INT PRIMARY KEY,
    Nome VARCHAR(100),
    Cargo VARCHAR(50),
    Salario DECIMAL(10, 2),
    Sexo CHAR(1)
);
''')

cur.execute('''
CREATE TABLE Garcom (
    ID_Funcionario INT PRIMARY KEY,
    Setor VARCHAR(100),
    FOREIGN KEY (ID_Funcionario) REFERENCES Funcionario(ID_Funcionario)
);
''')

cur.execute('''
CREATE TABLE Atendente (
    ID_Funcionario INT PRIMARY KEY,
    Bar VARCHAR(100),
    FOREIGN KEY (ID_Funcionario) REFERENCES Funcionario(ID_Funcionario)
);
''')

cur.execute('''
CREATE TABLE Mesas (
    ID_Mesa INT PRIMARY KEY,
    Numero INT
);
''')

cur.execute('''
CREATE TABLE Bebidas (
    ID_Bebida INT PRIMARY KEY,
    Nome VARCHAR(100),
    Preco DECIMAL(10, 2),
    Tipo VARCHAR(50),
    Estoque INT
);
''')

cur.execute('''
CREATE TABLE Comanda (
    ID_Comanda INT PRIMARY KEY
);
''')

cur.execute('''
CREATE TABLE Participa (
    CPF INT,
    ID_Festa INT,
    PRIMARY KEY (CPF, ID_Festa),
    FOREIGN KEY (CPF) REFERENCES Cliente(CPF),
    FOREIGN KEY (ID_Festa) REFERENCES Festa(ID_Festa)
);
''')

cur.execute('''
CREATE TABLE Reserva (
    CPF INT,
    ID_Mesa INT,
    PRIMARY KEY (CPF, ID_Mesa),
    FOREIGN KEY (CPF) REFERENCES Cliente(CPF),
    FOREIGN KEY (ID_Mesa) REFERENCES Mesas(ID_Mesa)
);
''')

cur.execute('''
CREATE TABLE Pagamento (
    CPF INT,
    ID_Comanda INT,
    PRIMARY KEY (CPF, ID_Comanda),
    FOREIGN KEY (CPF) REFERENCES Cliente(CPF),
    FOREIGN KEY (ID_Comanda) REFERENCES Comanda(ID_Comanda)
);
''')

cur.execute('''
CREATE TABLE Trabalha (
    ID_Funcionario INT,
    ID_Festa INT,
    PRIMARY KEY (ID_Funcionario, ID_Festa),
    FOREIGN KEY (ID_Funcionario) REFERENCES Funcionario(ID_Funcionario),
    FOREIGN KEY (ID_Festa) REFERENCES Festa(ID_Festa)
);
''')

cur.execute('''
CREATE TABLE Consumacao (
    ID_Comanda INT,
    ID_Bebida INT,
    PRIMARY KEY (ID_Comanda, ID_Bebida),
    FOREIGN KEY (ID_Comanda) REFERENCES Comanda(ID_Comanda),
    FOREIGN KEY (ID_Bebida) REFERENCES Bebidas(ID_Bebida)
);
''')

cur.execute('''
CREATE TABLE Debitar (
    ID_Comanda INT,
    ID_Mesa INT,
    PRIMARY KEY (ID_Comanda, ID_Mesa),
    FOREIGN KEY (ID_Comanda) REFERENCES Comanda(ID_Comanda),
    FOREIGN KEY (ID_Mesa) REFERENCES Bebidas(ID_Mesa)
);
''')

# Confirmar as operações no banco de dados
con.commit()



Inserindo dados na tabela Cliente

In [4]:
#exemplo
cur.execute('''
INSERT INTO Cliente (CPF, Nome, Telefone, Data_Nascimento)
VALUES ('123456789101', 'Gabriel Scaramussa', '5599999999', '2000-27-12')
''')

# Confirmar as operações no banco de dados
con.commit()


Consultando o SQL

In [5]:
# Executar uma consulta SQL para visualizar os dados da tabela Cliente
df = pd.read_sql_query("SELECT * FROM Cliente", con)

# Exibir os resultados da consulta
df



,CPF,Nome,Telefone,Data_Nascimento
0,123456789101,Gabriel Scaramussa,5599999999,2000-27-12


Inserindo funcionario

In [6]:
cur.execute('''
INSERT INTO Funcionario (ID_Funcionario, Nome, Cargo, Salario, Sexo)
VALUES (1, 'Angelo Viero', 'Garçom', 2500.00, 'M')
''')
con.commit()



Atualizar dados

In [11]:
# Atualizando nome e telefone de um cliente
cur.execute("""
UPDATE Cliente
SET Nome = ?, Telefone = ?
WHERE CPF = ?;
""", ('Maria Oliveira', '5597777777', '123456789101'))

# Atualizando salário de um funcionário
cur.execute("""
UPDATE Funcionario
SET Salario = ?
WHERE ID_Funcionario = ?;
""", (3200.00, 1))

# Confirmar as alterações no banco de dados
con.commit()


Consultando o SQL

In [7]:
# Executar uma consulta SQL para visualizar os dados da tabela Funcionario
df = pd.read_sql_query("SELECT * FROM Funcionario", con)

# Exibir os resultados da consulta
df

,ID_Funcionario,Nome,Cargo,Salario,Sexo
0,1,Angelo Viero,Garçom,2500,M


Deletando Dados

In [12]:
# Deletando um cliente pelo CPF
cur.execute("""
DELETE FROM Cliente
WHERE CPF = ?;
""", ('123456789101',))

# Deletando um funcionário pelo ID
cur.execute("""
DELETE FROM Funcionario
WHERE ID_Funcionario = ?;
""", (1,))

# Confirmar as exclusões no banco de dados
con.commit()


Teste de interface

In [13]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
#SOCORROOOOOO N SEI MONTAR INTERFACEEEEEEEEE


# Funções para operações no banco
def exibir_clientes():
    cur.execute("SELECT * FROM Cliente")
    rows = cur.fetchall()
    tabela.delete(*tabela.get_children())
    for row in rows:
        tabela.insert("", "end", values=row)

def adicionar_cliente():
    cpf = entrada_cpf.get()
    nome = entrada_nome.get()
    telefone = entrada_telefone.get()
    data_nascimento = entrada_data.get()
    try:
        cur.execute("INSERT INTO Cliente (CPF, Nome, Telefone, Data_Nascimento) VALUES (?, ?, ?, ?)",
                    (cpf, nome, telefone, data_nascimento))
        con.commit()
        exibir_clientes()
        messagebox.showinfo("Sucesso", "Cliente adicionado com sucesso!")
    except sqlite3.IntegrityError:
        messagebox.showerror("Erro", "CPF já existe ou erro nos dados!")



# Criar a janela principal
janela = tk.Tk()
janela.title("Interface de Banco de Dados")
janela.geometry("800x600")

# Seção para adicionar cliente
frame_adicionar = tk.Frame(janela)
frame_adicionar.pack(pady=20)

tk.Label(frame_adicionar, text="CPF").grid(row=0, column=0)
entrada_cpf = tk.Entry(frame_adicionar)
entrada_cpf.grid(row=0, column=1)

tk.Label(frame_adicionar, text="Nome").grid(row=1, column=0)
entrada_nome = tk.Entry(frame_adicionar)
entrada_nome.grid(row=1, column=1)

tk.Label(frame_adicionar, text="Telefone").grid(row=2, column=0)
entrada_telefone = tk.Entry(frame_adicionar)
entrada_telefone.grid(row=2, column=1)

tk.Label(frame_adicionar, text="Data de Nascimento").grid(row=3, column=0)
entrada_data = tk.Entry(frame_adicionar)
entrada_data.grid(row=3, column=1)

btn_adicionar = tk.Button(frame_adicionar, text="Adicionar Cliente", command=adicionar_cliente)
btn_adicionar.grid(row=4, column=0, columnspan=2, pady=10)

# Tabela para exibir clientes
frame_tabela = tk.Frame(janela)
frame_tabela.pack(pady=20)

colunas = ("CPF", "Nome", "Telefone", "Data de Nascimento")
tabela = ttk.Treeview(frame_tabela, columns=colunas, show="headings")
for col in colunas:
    tabela.heading(col, text=col)
    tabela.column(col, width=150)
tabela.pack()

btn_exibir = tk.Button(janela, text="Exibir Clientes", command=exibir_clientes)
btn_exibir.pack(pady=10)

# Iniciar o loop da interface
janela.mainloop()


TclError: no display name and no $DISPLAY environment variable